In [1]:
import lightgbm as lgb
from hyperopt import STATUS_OK, Trials, hp, space_eval, tpe, fmin



In [2]:
##Hyperopt

In [3]:
from sklearn.metrics import auc
from collections import Counter
def calculate_mAP(preds,label):
    ## mAP calculation
    AP = []
    num_class = 10
    predict_label_count_dict = Counter(preds)
    predict_label_count_dict = dict(sorted(predict_label_count_dict.items()))

    # For each class
    for c, freq in predict_label_count_dict.items() :
        TP = 0
        FN = 0

        temp_precision = []
        temp_recall = []

        for i in range(len(preds)):
            # Calculate TP and FN
            if label[i] == c and preds[i] == c :
                TP += 1

            elif label[i] != c and preds[i] == c :
                FN += 1

            # Calculate precision and recall
            if TP+FN != 0:
                temp_precision.append(TP/(TP+FN))
                temp_recall.append(TP/freq)

        # Save the AP value of each class to AP array
        AP.append(auc(temp_recall, temp_precision))

    # Calculate mAP
    mAP = sum(AP) / num_class

    return mAP

In [4]:
import lightgbm as lgb
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from hyperopt import STATUS_OK, Trials, hp, space_eval, tpe, fmin
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from collections import Counter

In [7]:
!pip install hyperopt

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# 데이터 로드
train = pd.read_csv('/content/drive/MyDrive/124000_augmented.csv')
train.isnull().any().sum()
test = pd.read_csv('/content/drive/MyDrive/public_test_data.csv')

In [7]:
print(train.shape)
print(test.shape)

(124000, 785)
(10000, 785)


In [8]:
df_train = train.copy()
df_test = test.copy()

In [9]:
X_train= df_train.drop(['label'],axis = 1)
X_label = df_train['label']
y_test = df_test.drop(['label'],axis = 1)
y_label = df_test['label']

X_train = X_train.astype('float32')
y_test = y_test.astype('float32')


In [10]:
"""
public_test_label.txt -> y_label
필요한 경우에 실행
"""

with open('label.txt', 'r') as file:
    lines = file.readlines()

series_data = pd.Series([int(line.strip().split()[1]) if line.strip().split()[1].isdigit() else 0 for line in lines], name='label', dtype='int64')
y_label = series_data

In [11]:
X_label = X_label.values # change to array for mAP
y_label = y_label.values # change to array for mAP

In [14]:
!pip install optuna


In [ ]:
import optuna
from optuna.integration import LightGBMPruningCallback
from optuna import Trial, create_study
from sklearn.model_selection import cross_val_score

# 로깅 설정
optuna.logging.set_verbosity(optuna.logging.INFO)

def objective(trial: Trial):
    params = {
        'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'num_leaves': trial.suggest_int('num_leaves', 2, 100),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'min_gain_to_split': trial.suggest_loguniform('min_gain_to_split', 0.1, 1.0),
        'max_bin': trial.suggest_int('max_bin', 128, 512),
        'scale_pos_weight': trial.suggest_uniform('scale_pos_weight', 1.0, 100.0),
        'objective': 'multiclass',
        'num_class': 10,
        'random_state': 45,
        'verbosity': -1,
        'metric': 'multi_logloss'
    }

    model = lgb.LGBMClassifier(**params)

    pca = PCA(n_components=700, random_state=45)
    lgbm_pipe = Pipeline([
        ('pca', pca),
        ('lgbm', model)
    ])

    train_x, valid_x, train_y, valid_y = train_test_split(X_train, X_label, test_size=0.3, stratify=X_label)

    lgbm_pipe.fit(train_x, train_y)

    preds = lgbm_pipe.predict(valid_x)

    mAP = calculate_mAP(preds, valid_y)

    return mAP

# Callback 함수 정의
def print_param(study, trial):
    print("Trial {} params: {}".format(trial.number, trial.params))

study = create_study(direction="maximize", pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=100, callbacks=[print_param], show_progress_bar=True)

best_params = study.best_params

print('Best trial: score {}, params {}'.format(study.best_value, best_params))


[I 2023-11-30 15:04:55,286] A new study created in memory with name: no-name-8a1677b3-bf52-439a-a3f3-3834c4e5816d


  0%|          | 0/100 [00:00<?, ?it/s]

<ipython-input-12-f0b5f67d9f3d>:11: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.3),
<ipython-input-12-f0b5f67d9f3d>:14: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
<ipython-input-12-f0b5f67d9f3d>:15: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
<ipython-input-12-f0b5f67d9f3d>:18: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This featu

[I 2023-11-30 15:09:04,226] Trial 0 finished with value: 0.8535687068354122 and parameters: {'learning_rate': 0.18969656382927796, 'max_depth': 5, 'num_leaves': 28, 'feature_fraction': 0.7180625251224497, 'bagging_fraction': 0.6271814580704598, 'bagging_freq': 3, 'min_child_samples': 94, 'lambda_l1': 3.067253387681708e-08, 'lambda_l2': 2.9031496199429708e-06, 'min_gain_to_split': 0.18707053466105983, 'max_bin': 341, 'scale_pos_weight': 63.33787294936572}. Best is trial 0 with value: 0.8535687068354122.
Trial 0 params: {'learning_rate': 0.18969656382927796, 'max_depth': 5, 'num_leaves': 28, 'feature_fraction': 0.7180625251224497, 'bagging_fraction': 0.6271814580704598, 'bagging_freq': 3, 'min_child_samples': 94, 'lambda_l1': 3.067253387681708e-08, 'lambda_l2': 2.9031496199429708e-06, 'min_gain_to_split': 0.18707053466105983, 'max_bin': 341, 'scale_pos_weight': 63.33787294936572}


<ipython-input-12-f0b5f67d9f3d>:11: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.3),
<ipython-input-12-f0b5f67d9f3d>:14: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
<ipython-input-12-f0b5f67d9f3d>:15: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
<ipython-input-12-f0b5f67d9f3d>:18: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This featu

...

<ipython-input-12-f0b5f67d9f3d>:11: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.3),
<ipython-input-12-f0b5f67d9f3d>:14: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
<ipython-input-12-f0b5f67d9f3d>:15: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
<ipython-input-12-f0b5f67d9f3d>:18: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This featu

[I 2023-11-30 16:27:22,390] Trial 19 finished with value: 0.8768369901783185 and parameters: {'learning_rate': 0.2105063738406757, 'max_depth': 7, 'num_leaves': 44, 'feature_fraction': 0.6321950149577108, 'bagging_fraction': 0.8086977751211619, 'bagging_freq': 2, 'min_child_samples': 32, 'lambda_l1': 0.06239170104946315, 'lambda_l2': 0.0001927034824133381, 'min_gain_to_split': 0.12807985798558277, 'max_bin': 289, 'scale_pos_weight': 46.591689947898594}. Best is trial 19 with value: 0.8768369901783185.
Trial 19 params: {'learning_rate': 0.2105063738406757, 'max_depth': 7, 'num_leaves': 44, 'feature_fraction': 0.6321950149577108, 'bagging_fraction': 0.8086977751211619, 'bagging_freq': 2, 'min_child_samples': 32, 'lambda_l1': 0.06239170104946315, 'lambda_l2': 0.0001927034824133381, 'min_gain_to_split': 0.12807985798558277, 'max_bin': 289, 'scale_pos_weight': 46.591689947898594}


<ipython-input-12-f0b5f67d9f3d>:11: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.3),
<ipython-input-12-f0b5f67d9f3d>:14: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
<ipython-input-12-f0b5f67d9f3d>:15: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
<ipython-input-12-f0b5f67d9f3d>:18: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This featu

[I 2023-11-30 16:30:34,635] Trial 20 finished with value: 0.8256041799828793 and parameters: {'learning_rate': 0.15104018169601616, 'max_depth': 4, 'num_leaves': 46, 'feature_fraction': 0.6199333611319688, 'bagging_fraction': 0.7558467670567142, 'bagging_freq': 2, 'min_child_samples': 27, 'lambda_l1': 0.07743997959326938, 'lambda_l2': 2.0564542050773438e-06, 'min_gain_to_split': 0.13209339810252, 'max_bin': 283, 'scale_pos_weight': 47.13488936177953}. Best is trial 19 with value: 0.8768369901783185.
Trial 20 params: {'learning_rate': 0.15104018169601616, 'max_depth': 4, 'num_leaves': 46, 'feature_fraction': 0.6199333611319688, 'bagging_fraction': 0.7558467670567142, 'bagging_freq': 2, 'min_child_samples': 27, 'lambda_l1': 0.07743997959326938, 'lambda_l2': 2.0564542050773438e-06, 'min_gain_to_split': 0.13209339810252, 'max_bin': 283, 'scale_pos_weight': 47.13488936177953}


<ipython-input-12-f0b5f67d9f3d>:11: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.3),
<ipython-input-12-f0b5f67d9f3d>:14: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
<ipython-input-12-f0b5f67d9f3d>:15: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
<ipython-input-12-f0b5f67d9f3d>:18: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This featu

In [ ]:
# 시각화
optuna.visualization.plot_optimization_history(study)

In [ ]:
# 파라미터들관의 관계
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
# 하이퍼파라미터 중요도
optuna.visualization.plot_param_importances(study)